### I Reddit Data Collection

#### What's Reddit flair?

Flair is a sort of 'tag' that can be added to posts or usernames within a sub-reddit. A Subreddit is dedicated to a particular topic that people write about, and they're denoted by /r/, followed by the subreddit's name, e.g., /r/india.

#### Where and How Collect reddit data?

The task is to perform data aquisition of reddit posts from the /india subreddit, classification of the posts into 12 different flairs and deploying the best model as a web service.
1. Fetched 52,000 (1,000 per week) for the last year, India subreddit data for each of the 12 flairs using Pushshift API.
2. The data includes title, comments, selftext, url, number of comments, score, link_flair_text.
3. Many post do not have a flair associated with them, they will be removed from the dataset.
4. Unfortunately the posts have not been tagged with their comments in the dataset. To extract this information, PRAW is used.

#### How data turned out imbalanced?

Ideally classification of the posts as soon as it is created mostly uses the title , comments , url of the post as initial inputs to the models.After the first exploration of basic features on different models performing flair classification till Part- 3 , the classifiers wer'nt equipped to handle imbalanced classes and were biased towards majority classes. This has become counter-productive in the lieu of aiming only for high accuracy on the imbalanced dataset.So, the Part-1 has been performed again by balancing the current imbalanced dataset.

#### How important it is to have a balanced dataset?

It is important to understand the relevance of accuracy metrics when evaluating the performance of a model trained on imbalanced data. If you want to build a predictive model, particularly one that performs well and plan to use a basic ML framework (i.e. Random Forest, SVM, Naive Bayes, etc), then a balanced dataset offers advantages in terms of training speed and accuracy.  Much of the literature on class imbalance finds that random undersampling (down sampling the majority class to the size of the minority class) can drive performance gains.

#### How to balance this dataset?

To build a balanced dataset, we will limit the number of samples for each flair at 200 and further randomly sample from the previously acquired dataset ensuring that each post now contains non null value of title, url, self_text and comments.


### II EDA

I. Since the Datset used was unbalanced , a lot of data visualisation metrics have been used to inspect the data patterns. The unbalanced dataset consists of the following columns :


| Label  | Feature  | Feature Description  |
|:-:|:-:|:-:|
| 1  | title  | Title of the post  |
| 2  | url  | URL associated with the post  |
| 3  | selftext  | description of the post  |
| 4  | comments  | list of top comments  |
| 5  | num_comments  | number of comments on the post  |
| 6  | link_flair_text	  | flair of the post  |

After a brief inspection of the data , 12 flairs are being considered. The number of samples per set is:

| Label  | Flair  | Occurences  |
|---|:-:|---|
| 1  | AskIndia  |  4569 |
| 2  | Non-Political  | 7102  |
| 3  | [R]eddiquette  | 760  |
| 4  | Photography  | 847  |
| 5  | Science/Technology | 1340 |
| 6  | Politics  | 7798  |
| 7  | Business/Finance  | 1886  |
| 8  | Policy/Economy  | 1403  |
| 9  | Sports  | 468  |
| 10  | Food  | 411  |
| 11  | AMA  | 2  |
| 12  | Coronavirus  | 2092  |

II. Started with basic data exploration and comparision like :
* Number of samples we have for each flair category
* Distribution of no of words in title
* Average length of comments per post
* score vs num_comment (To depict gaining Polpularity)
* max and min length of the comments in each flair category

III. Initially experimented words as individual units, and considered their relationships.As its text data, correlation among words must be measured in a binary form rather than using insignificant coefficients.For this task, many interesting text analyses are there for calculating and visualizing relationships between words in the text dataset. This includes the n grams, which tokenizes by pairs of adjacent words rather than by individual ones and detected the most common features of the corpora.The main intution behind performing these data analyses is a pair of consecutive words might capture structure that isn’t present when one is just counting single words, and may provide context that makes tokens more understandable.

IV. Finally, explored the topic modeling algorithm to this data set, to see whether it would provide any benefit, and fit with what we are doing for our features i.e., Latent Semantic Analysis (LSA) technique in topic modeling.
The core idea behind this is to :
* the similarity of different words
* the similarity of terms (or “queries”)which becomes useful in information retrieval, when we want to retrieve passages most relevant to our search query.
* its easy and efficient but it lacks interpretable embeddings (we don’t know what the topics are)

### III Building a Flair Detector

I.  After we have our features, we can train a classifier to try to predict the tag of a post.But its neccessary to perform data cleaning for removing unwanted complexities later on. So, after thorough research on how to perform this task , have followed this [article](https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568) which gives a brief idea on how to use vectorizers and pipeline classes.An extensive research in terms of parameters and various other eval metrics is needed for models to produce general hypotheses, which then make the most accurate predictions possible about future instances.For this balanced data set, our text cleaning step includes removing stop words, change text to lower case, remove punctuation, remove bad characters, and so on.

II. Features used as a combination of original features for performance gains:


a) ```title```

b) ```comments+title+url```

c) ```selftext+url+title```

d) ```title+comments+selftext+url```

II. After data cleaning, ML algortithms listed as follows have been experimented under valid reasons:


| Label | Classifier  | Accuracy |
|:-:|---|---|
| 1 | Naive-Bayes  | 57  |
| 2 | Linear SVM  | 74  |
| 3 | Logistic Regression  | 74 |
| 4 | Random Forest  | 81  |
| 5 | MLP  | 58  |
| 6 | BOW with Keras  | 61 |

III. The best performer out of all the classifiers tried is Random Forest. Initially, this model have given accuracy which was just average of all but after experimenting and fine tuning the parameters in depth, the highest level of the accuracy achieved was 81.
* Started with a Naive Bayes classifier, which provides a nice baseline for this task with the help of Pipeline class in Scilkit-Learn.
* Linear SVM was regarded widely as best text classification algorithm.This acheived accuracy which was 13% improvement over Naive-Bayes classifier.
* Logistic regression is a simple and easy to understand classification algorithm, and can be easily generalized to multiple classes.This is the second best model with 72% accuracy on this multi-class text classification data set.
* The Multi Layer Perceptron (MLP) algorithm with 51% accuracy performs least of all because of the too many parameters as it is fully connected.This algorithm lags because of the way it is trained, it can not guarantee that the minima it stops at during training is the global minima. The MLP algorithm can, therefore, get stuck in a local minima.
* Finally, we use BOW(Bag of Words) with Keras which is a Python Deep Learning library has acheived 61%.Its simple but sometimes expensive to compute especially when contextual info aren’t relevant.

IV. Therefore, Random Forest is used as the final model with 79% accuracy with a feature combination of title+url+selftext+comments

V. **Why not use state-of-the-art models like BERT/LSTM/GRU?**
BERT which has caused a stir in ML community by presenting state-of-the-art results in a wide variety of NLP tasks,makes use of Transformer, an attention mechanism that learns contextual relations between words (or sub-words) in a text.Potential issues can be while using pre trained NLP based models like this are:
* vocab_size. When using Hinglish data (Hindi text written using English Alphabets) there can be new words which are not present in English vocabulary.
* Models like BERT can be of significant size and because of restrictions of Heroku and Github, it becomes difficult to deploy.

### IV Building a Web App and it's deployment on Heroku

#### Making the web app using Flask

For the web app, Flask in used. A simple form is created which accepts the link to the Reddit post.
PRAW is used to fetch the data from given url and run through the model to predict the flair. The endpoint
```/automated_testing``` has been created to test using a text file which return a JSON file in response.